In [9]:
# Import libraries
import mailbox
import bs4
import email
from email import message
import quopri,base64
import pandas as pd
import sys
sys.path.append('..')

In [10]:
mbox_data = mailbox.mbox('../Data/Destacados.mbox')
num_emails = len(mbox_data)
num_emails
# for idx, email_data in enumerate(mbox_data):
#     print('Parsing email {0} of {1}'.format(idx, num_emails))

4

In [11]:
# # Check some info
to_list = []
for i in list(range(len(mbox_data))):
    to_list.append(mbox_data[i]["To"])

subject_list = []
for i in list(range(len(mbox_data))):
     subject_list.append(mbox_data[i]["Subject"])

from_list = []
for i in list(range(len(mbox_data))):
     from_list.append(mbox_data[i]["From"])

date_list = []
for i in list(range(len(mbox_data))):
     date_list.append(mbox_data[i]["Date"])
        
print(to_list)
print(subject_list)
print(from_list)
print(date_list)

['alessiobunsetti@gmail.com', 'alessiobunsetti@gmail.com', 'alessiobunsetti@gmail.com', 'alessiobunsetti@gmail.com']
['Fwd: 105', 'Fwd: 105', 'Fwd: 105', 'Fwd: 105']
['Javier Chamorro <javier.chamorro.w@gmail.com>', 'Javier Chamorro <javier.chamorro.w@gmail.com>', 'Javier Chamorro <javier.chamorro.w@gmail.com>', 'Javier Chamorro <javier.chamorro.w@gmail.com>']
['Fri, 5 Nov 2021 16:08:16 +0100', 'Fri, 5 Nov 2021 16:08:35 +0100', 'Fri, 5 Nov 2021 16:08:26 +0100', 'Fri, 5 Nov 2021 12:45:34 +0100']


In [12]:
body_list = []

for msg in mbox_data:
    if msg.is_multipart():    #Walk through the parts of the email to find the text body.
        for part in msg.walk():
            if part.is_multipart(): # If part is multipart, walk through the subparts.
                for subpart in part.walk():
                    if subpart.get_content_type() == 'text/plain':
                        body = subpart.get_payload(decode=False) # Get the subpart payload (i.e the message body)
                        body_list.append(body)
                    for k,v in subpart.items():
                            if k == 'Content-Transfer-Encoding':
                                cte = v             # Keep the Content Transfer Encoding
            elif subpart.get_content_type() == 'text/plain':
                body = part.get_payload(decode=False)           # part isn't multipart Get the payload
                for k,v in part.items():
                    if k == 'Content-Transfer-Encoding':
                        cte = v                      # Keep the Content Transfer Encoding

# body_list

In [13]:
body_clean = []
for i, b in enumerate(body_list):
    index = body_list[i].find('Welches')
    clean = body_list[i][index:]
    clean = clean.replace('=C3=A4','a')
    clean = clean.replace('=C3=B6','o')
    clean = clean.replace('=C3=BC','u')
    clean = clean.replace('=C3=9F','B') #ß
    clean = clean.replace('Datenschutzerkla=\r\nrung\r\ngelesen','datenschutzerklarung gelesen')
    clean = clean.replace('gelesen ha=\r\nbe\r\nund akzeptiere','gelesen habe und akzeptiere')
        
    body_clean.append(clean)
# body_clean

In [14]:
body_clean

['Welches Tesla Modell mochtest Du verkaufen?*\r\nModel 3\r\n*Ausweisbare MWST?*\r\nJa\r\n*Datum der Erstzulassung*\r\n6363636366\r\n*Kilometerstand*\r\nAusgewahlter Wert: 11252 (0 min / 12000 max)\r\n*Antriebsversion*\r\nMaximale Reichweite\r\n*Lackierungen*\r\nRed Multi-Coat\r\n*Rader*\r\nModel 3 - 18" Aero Felgen\r\n*Innenansicht*\r\nKomplett schwarz\r\n*Autopilot*\r\nVolles Potential fur autonomes Fahren\r\n*Unfallfahrzeug:*\r\nJa\r\n*Brutto-Listenpreis*\r\n999999\r\n*Name*\r\nhhh ddd\r\n*Strasse und Hausnummer*\r\nddd\r\n*PLZ und Ort*\r\nddd\r\n*Telefonische Erreichbarkeit*\r\n010101100101\r\n*E-Mail*\r\nhhhh@hhhh.de\r\n*Datenschutzerklarung*\r\nHiermit bestatige ich, dass ich die nachfolgende datenschutzerklarung gelesen habe und akzeptiere.\r\n*Ankaufs AGB*\r\nHiermit bestatige ich, dass ich die nachfolgende datenschutzerklarung gelesen habe und akzeptiere.\r\nGesendet von sellyourtesla.de\r\n',
 'Welches Tesla Modell mochtest Du verkaufen?*\r\nModel 3\r\n*Ausweisbare MWST?*\r\n

In [15]:
question_1 = 'Welches Tesla Modell mochtest Du verkaufen?'
question_2 = 'Ausweisbare MWST?'
question_3 = 'Datum der Erstzulassung'
question_4 = 'Kilometerstand'
question_5 = 'Antriebsversion'
question_6 = 'Lackierungen'
question_7 = 'Rader'
question_8 = 'Innenansicht'
question_9 = 'Autopilot'
question_10 = 'Unfallfahrzeug:'
question_11 = 'Beschadigungen'
question_12 = 'Zubehor'
question_13 = 'Brutto-Listenpreis'
question_14 = 'Name'
question_15 = 'Strasse und Hausnummer'
question_16 = 'PLZ und Ort'
question_17 = 'Telefonische Erreichbarkeit'
question_18 = 'E-Mail'
question_19 = 'Datenschutzerklarung'
question_20 = 'Ankauf AGB'
question_21 = 'Ankaufs AGB'

questions = [question_1, question_2, question_3, question_4, question_5, question_6, question_7, question_8, question_9, question_10, question_11, question_12, question_13, question_14, question_15, question_16, question_17, question_18, question_19, question_20, question_21]

questions_len = []
for question in questions:
    question_len = len(question)
    questions_len.append(question_len)
# questions_len
next_question = []
for i in list(range(len(questions))):
    if i == (len(questions)-1):
        next_question.append('esendet von sellyourtesla.d')
    else:
        next_q = questions[(i+1)]
        next_question.append(next_q)
next_question

['Ausweisbare MWST?',
 'Datum der Erstzulassung',
 'Kilometerstand',
 'Antriebsversion',
 'Lackierungen',
 'Rader',
 'Innenansicht',
 'Autopilot',
 'Unfallfahrzeug:',
 'Beschadigungen',
 'Zubehor',
 'Brutto-Listenpreis',
 'Name',
 'Strasse und Hausnummer',
 'PLZ und Ort',
 'Telefonische Erreichbarkeit',
 'E-Mail',
 'Datenschutzerklarung',
 'Ankauf AGB',
 'Ankaufs AGB',
 'esendet von sellyourtesla.d']

In [16]:
answers_list = []

for body_n in body_clean:
    answers = []
    iteration_count = 0
    for i, x, s in zip(questions_len, questions, next_question):
        iteration_count += 1
        if x != questions[len(questions)-1]:    #mira si es la última pregunta
            index = body_n.find(x)
            if index != -1:                     #mira si encuentra la pregunta
                index_plus_one = body_n.find(s)
                index_plus_two = body_n.find(next_question[iteration_count])
                

                if index_plus_one != -1:        #mira si encuentra la siguiente pregunta
                    answer = body_n[(index+i):(index_plus_one)]
                    answers.append(answer[3:-3])
                elif (index_plus_one == -1) and (index_plus_two != -1): #si no encuentra la siguiente, mira una mas alla
                    index_plus_two = body_n.find(next_question[iteration_count])
                    answer = body_n[(index+i):(index_plus_two)]
                    answers.append(answer[3:-3])
                    print(x, '1')
                    print(answer[3:-3])

                elif (index_plus_one == -1) and (index_plus_two == -1):   #si no encuentra dos preguntas mas alla
                    try:                                                  #intenta mirar una tercera
                        index_plus_three = body_n.find(next_question[iteration_count+1]) 
                        if index_plus_three != -1:
                            answer = body_n[(index+i):(index_plus_three)]
                            answers.append(answer[3:-3])
                            print(x, 'try')
                            print(answer[3:-3])
                        else:
                            answers.append('Las siguientes 3 columnas no coinciden, no se donde termina la respuesta')
                            print(x, 'except')
                            print('WTF')
                        
                    except:                                               #si no es posible imprime
                        answers.append('WTF')
                        print(x, 'except')
                        print('WTF')

            else:
                answers.append('N/A')

        elif x == questions[len(questions)-1]:    #en el caso de ser la última pregunta
            index = body_n.find(x)
            if index != -1:
                end_of_body = len(body_n)
                fin_esendet = body_n.find(next_question[len(next_question)-1])
                answer = body_n[(index+i):(fin_esendet)] #última respuesta va desde el fin de la pregunta hasta esendet von sellyourtesla.d
                answers.append(answer[3:-3])
                print(x, 'ult')
                print(answer[3:-3])
            else:
                answers.append('N/A')
                print(x, 'ult')
                print('N/A')
    
    answers_list.append(answers)
answers_list

Unfallfahrzeug: try
Ja
Datenschutzerklarung 1
Hiermit bestatige ich, dass ich die nachfolgende datenschutzerklarung gelesen habe und akzeptiere.
Ankaufs AGB ult
Hiermit bestatige ich, dass ich die nachfolgende datenschutzerklarung gelesen habe und akzeptiere.
Unfallfahrzeug: try
Nein
Ankauf AGB 1
Hiermit bestatige ich, dass ich die nachfolgende Ankauf AGB gelesen habe und akzeptiere.
Ankaufs AGB ult
N/A
Ankauf AGB 1
Hiermit bestatige ich, dass ich die nachfolgende Ankauf AGB gelesen habe und akzeptiere.
Ankaufs AGB ult
N/A
Autopilot except
WTF
Datenschutzerklarung try
Hiermit bestatige ich, dass ich die nachfolgende datenschutzerklarung gelesen habe und akzeptiere.
Ankaufs AGB ult
N/A


[['Model 3',
  'Ja',
  '6363636366',
  'Ausgewahlter Wert: 11252 (0 min / 12000 max)',
  'Maximale Reichweite',
  'Red Multi-Coat',
  'Model 3 - 18" Aero Felgen',
  'Komplett schwarz',
  'Volles Potential fur autonomes Fahren',
  'Ja',
  'N/A',
  'N/A',
  '999999',
  'hhh ddd',
  'ddd',
  'ddd',
  '010101100101',
  'hhhh@hhhh.de',
  'Hiermit bestatige ich, dass ich die nachfolgende datenschutzerklarung gelesen habe und akzeptiere.',
  'N/A',
  'Hiermit bestatige ich, dass ich die nachfolgende datenschutzerklarung gelesen habe und akzeptiere.'],
 ['Model 3',
  'Ja',
  '16.5.2021',
  'Ausgewahlter Wert: 9429 (0 min / 12000 max)',
  'Maximale Reichweite',
  'Pearl White Multi-Coat',
  'Model 3 - 18" Aero Felgen',
  'Komplett schwarz',
  'Basis',
  'Nein',
  'N/A',
  'N/A',
  '55000',
  'Gunter Meisinger',
  '94116 Hutthurm',
  'Hutthurm',
  '015115617854',
  'g.meisinger@mlogica.de',
  'Hiermit bestatige ich, dass ich die nachfolgende datenschutzerklarung gelesen habe und akzeptiere.',
  

In [17]:
next_question[len(next_question)-1]

'esendet von sellyourtesla.d'

In [19]:
df = pd.DataFrame(answers_list)
df.columns=questions
pd.set_option('display.max_columns', None)
print(to_list)
print(subject_list)
print(from_list)
print(date_list)
df['Date'] = date_list
df['Subject'] = subject_list
df['From'] = from_list
df['To'] = to_list
df

['alessiobunsetti@gmail.com', 'alessiobunsetti@gmail.com', 'alessiobunsetti@gmail.com', 'alessiobunsetti@gmail.com']
['Fwd: 105', 'Fwd: 105', 'Fwd: 105', 'Fwd: 105']
['Javier Chamorro <javier.chamorro.w@gmail.com>', 'Javier Chamorro <javier.chamorro.w@gmail.com>', 'Javier Chamorro <javier.chamorro.w@gmail.com>', 'Javier Chamorro <javier.chamorro.w@gmail.com>']
['Fri, 5 Nov 2021 16:08:16 +0100', 'Fri, 5 Nov 2021 16:08:35 +0100', 'Fri, 5 Nov 2021 16:08:26 +0100', 'Fri, 5 Nov 2021 12:45:34 +0100']


,Welches Tesla Modell mochtest Du verkaufen?,Ausweisbare MWST?,Datum der Erstzulassung,Kilometerstand,Antriebsversion,Lackierungen,Rader,Innenansicht,Autopilot,Unfallfahrzeug:,Beschadigungen,Zubehor,Brutto-Listenpreis,Name,Strasse und Hausnummer,PLZ und Ort,Telefonische Erreichbarkeit,E-Mail,Datenschutzerklarung,Ankauf AGB,Ankaufs AGB,Date,Subject,From,To
0,Model 3,Ja,6363636366,Ausgewahlter Wert: 11252 (0 min / 12000 max),Maximale Reichweite,Red Multi-Coat,"Model 3 - 18"" Aero Felgen",Komplett schwarz,Volles Potential fur autonomes Fahren,Ja,N/A,N/A,999999,hhh ddd,ddd,ddd,010101100101,hhhh@hhhh.de,"Hiermit bestatige ich, dass ich die nachfolgen...",N/A,"Hiermit bestatige ich, dass ich die nachfolgen...","Fri, 5 Nov 2021 16:08:16 +0100",Fwd: 105,Javier Chamorro <javier.chamorro.w@gmail.com>,alessiobunsetti@gmail.com
1,Model 3,Ja,16.5.2021,Ausgewahlter Wert: 9429 (0 min / 12000 max),Maximale Reichweite,Pearl White Multi-Coat,"Model 3 - 18"" Aero Felgen",Komplett schwarz,Basis,Nein,N/A,N/A,55000,Gunter Meisinger,94116 Hutthurm,Hutthurm,015115617854,g.meisinger@mlogica.de,"Hiermit bestatige ich, dass ich die nachfolgen...","Hiermit bestatige ich, dass ich die nachfolgen...",N/A,"Fri, 5 Nov 2021 16:08:35 +0100",Fwd: 105,Javier Chamorro <javier.chamorro.w@gmail.com>,alessiobunsetti@gmail.com
2,Model 3,Ja,21.04.2020,Ausgewahlter Wert: 2847 (0 min / 12000 max),Standard Plus,Pearl White Multi-Coat,"Model 3 - 18"" Aero Felgen",Komplett schwarz,Basis,Ja,Test Test,Anhangerkupplung,Test,Test Test,Test,Test,28971309827,test@test.de,"Hiermit bestatige ich, dass ich die nachfolgen...","Hiermit bestatige ich, dass ich die nachfolgen...",N/A,"Fri, 5 Nov 2021 16:08:26 +0100",Fwd: 105,Javier Chamorro <javier.chamorro.w@gmail.com>,alessiobunsetti@gmail.com
3,Model Y,Ja,09.10.2021,Ausgewahlter Wert: 1432 (0 min / 12000 max),Performance,Red Multi-Coat,"Model Y - 20"" Induction Felgen",Schwarz und weiB,"Las siguientes 3 columnas no coinciden, no se ...",N/A,N/A,N/A,N/A,Justin Breschul,Sonderschulstrasse 1,42105 Wuppertal,0189765432,justin@gmail.com,"Hiermit bestatige ich, dass ich die nachfolgen...",N/A,N/A,"Fri, 5 Nov 2021 12:45:34 +0100",Fwd: 105,Javier Chamorro <javier.chamorro.w@gmail.com>,alessiobunsetti@gmail.com


df.to_csv(path_or_buf='../Data/mbox-output.csv', sep = ';', index=False)

df.to_csv(path_or_buf='../Data/parsed_emails.csv', index=False)
df.to_csv(path_or_buf='../Data/parsed_emails.csv', index=True)

data = df.reset_index(drop=True)
data

data.to_csv(path_or_buf='../Data/parsed_emails.csv', index=False)
